In [2]:
print("Hello VADER!")

Hello VADER!


In [44]:
import pandas as pd

filename = "output2020_02_sm.csv"

df = pd.read_csv(filename)

df.head(2).full_text

0    Wereldwijd groeien de zorgen om het #coronavir...
1    Of de aantallen kloppen niet. Of dit filmpje i...
Name: full_text, dtype: object

In [27]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

def classify_sentiment(sentence):
    """
    This function accepts a string and 
    """
    sid = SentimentIntensityAnalyzer()

    sentiment_dict = sid.polarity_scores(sentence)

    
    print(f"Overall sentiment dictionary is : {sentiment_dict}")
    
    print(f"sentence was rated as {sentiment_dict['neg']*100} % Negative")
    print(f"sentence was rated as {sentiment_dict['neu']*100} % Neutral")
    print(f"sentence was rated as {sentiment_dict['pos']*100} % Positive")
    
    if sentiment_dict["compound"] >= 0.05:
        print("Positive")
    elif sentiment_dict["compound"] <= -0.05:
        print("Negative")
    else:
        print("Neutral")

In [29]:
# Example with one sentence
sentence = "Wereldwijd groeien de zorgen om het #coronavirus. Ruim 11.000 mensen zijn inmiddels besmet in China, van wie er 258 zijn overleden."

classify_sentiment(sentence)

Overall sentiment dictionary is : {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
sentence was rated as 0.0 % Negative
sentence was rated as 100.0 % Neutral
sentence was rated as 0.0 % Positive
Neutral


In [52]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import re

stop_words = set(stopwords.words("english"))

def remove_urls(sentence):
    """
    This function removes urls from a String.
    """
    
    return re.sub(r"https?://\S*","",sentence,flags=re.MULTILINE)

def remove_symbols(sentence):     
    """
    This function removes symbols from a String.
    """
    return re.sub("[#,.!?:]", "",sentence)
    

def remove_stopwords(sentence):
    """
    This function removes english stopwords from a String.
    """
    
    word_tokens = word_tokenize(sentence)
    
    filtered_tokens = [w for w in word_tokens if not w.lower() in stop_words]
    
    filtered_sentence = " ".join(filtered_tokens)
    
    return filtered_sentence
    
    
sentence = "This is sentence is about #coronavirus and this news articale published at: https://stackabuse.com/python-convert-list-to-string/"

filtered_sentence = remove_urls(sentence)
print(filtered_sentence)
filtered_sentence = remove_symbols(filtered_sentence)
print(filtered_sentence)
filtered_sentence = remove_stopwords(filtered_sentence)
print(filtered_sentence)


This is sentence is about #coronavirus and this news articale published at: 
This is sentence is about coronavirus and this news articale published at 
sentence coronavirus news articale published


In [66]:
import pandas as pd
from nltk.corpus import stopwords
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from deep_translator import GoogleTranslator

filename = "output2020_02_lg.csv"

nl_to_en = GoogleTranslator(source='nl', target='en')

stop_words = set(stopwords.words("english"))

sid = SentimentIntensityAnalyzer()

df = pd.read_csv(filename)

df["processed_text"] = df["full_text"].apply(lambda txt: nl_to_en.translate(txt))

df["processed_text"] = df["processed_text"].apply(lambda txt: txt.lower())

df["processed_text"] = df["processed_text"].apply(lambda txt: " ".join([word for word in txt.split() if word not in stop_words]))

df["scores"] = df["processed_text"].apply(lambda txt: sid.polarity_scores(txt))

df

KeyboardInterrupt: 

In [98]:
import pandas as pd
from nltk.corpus import stopwords
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from deep_translator import GoogleTranslator
from deep_translator.exceptions import TooManyRequests
from logging import warning
from logging import info
from time import perf_counter

filename = "output2020_02_lg.csv"

nl_to_en = GoogleTranslator(source='auto', target='en')

df = pd.read_csv(filename)

texts = set(df["full_text"])


try:
    start = perf_counter()
    translated_texts = nl_to_en.translate_batch(texts)
    elapsed_time = perf_counter()-start
    print(f"Translation took: {elapsed_time} seconds.")
except TooManyRequests as e:
    warning(f"Error caught: {e.message}")

se = pd.Series(translated_texts)

df["processed_texts"] = se.values

df

ConnectionError: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: /m?tl=en&sl=auto&q=So+eerie+https%3A%2F%2Ft.co%2FULfWnjz4Xy (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fef5bafa580>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))

In [94]:
from deep_translator.exceptions import TooManyRequests
from logging import warning

def throw_error():
    
    raise TooManyRequests()
    
try:
    print("Try and raise an error.")
    throw_error()
    print("Failed to throw error.")
except TooManyRequests as e:
    warning(f"Error caught: {e.message}")

Try and raise an error.
